<a href="https://colab.research.google.com/github/Hemasri-data/Predicting-Recidivism-through-survival-models/blob/main/COMPARISON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sksurv.svm import FastKernelSurvivalSVM
from lifelines import WeibullAFTFitter
from sksurv.util import Surv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Load the dataset
file_path = "/content/FINAL FILE.csv"
df = pd.read_csv(file_path)

# Select relevant columns for survival analysis
numerical_features = ['age', 'priors_count', 'decile_score']  # Add more numerical features if available
categorical_features = ['gender', 'offense_type']  # Example categorical features if available
target_duration = 'duration'
target_event = 'event'

# Ensure target_duration is positive before creating Surv object
# This is the crucial step to address the ValueError
df[target_duration] = df[target_duration].apply(lambda x: 0.1 if x <= 0 else x)

# Convert to structured survival format
y = Surv.from_arrays(event=df[target_event].values, time=df[target_duration].values)
print(df.columns)
numerical_features = [col for col in ['age', 'priors_count', 'decile_score'] if col in df.columns]
categorical_features = [col for col in ['gender', 'offense_type'] if col in df.columns]
X = df[numerical_features + categorical_features]

# Handle missing values
X = X.fillna(X.median())

# Preprocessing: Scale numerical features and encode categorical features
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical_features),
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Survival SVM Model
svm_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", FastKernelSurvivalSVM())
])

# The target_duration column is already positive, so no need to modify it again
#y = Surv.from_arrays(event=df[target_event], time=df[target_duration])
svm_pipeline.fit(X_train, y_train)
predicted_risk_scores = svm_pipeline.predict(X_test)

# Train Weibull AFT Model using lifelines
aft_fitter = WeibullAFTFitter()
aft_data = X_train.copy()
aft_data[target_duration] = df.loc[X_train.index, target_duration]
aft_data[target_event] = df.loc[X_train.index, target_event]
aft_fitter.fit(aft_data, duration_col=target_duration, event_col=target_event)
predicted_survival_times = aft_fitter.predict_expectation(X_test)

# Visualization: Distribution of Predicted Risk Scores
plt.figure(figsize=(10, 5))
sns.histplot(predicted_risk_scores, bins=30, kde=True, color='blue', label='Survival SVM')
plt.xlabel("Predicted Risk Scores")
plt.ylabel("Count")
plt.title("Distribution of Predicted Risk Scores (Survival SVM)")
plt.legend()
plt.show()

# Visualization: Comparison of Predicted Survival Times
plt.figure(figsize=(10, 5))
sns.histplot(predicted_survival_times, bins=30, kde=True, color='red', label='Weibull AFT(lifelines)')
plt.xlabel("Predicted Survival Times")
plt.ylabel("Count")
plt.title("Comparison of Predicted Survival Times (Weibull AFT)")
plt.legend()
plt.show()

# Model Performance
svm_train_score = svm_pipeline.score(X_train, y_train)
svm_test_score = svm_pipeline.score(X_test, y_test)
print(f"Survival SVM - Training Score: {svm_train_score:.4f}")
print(f"Survival SVM - Testing Score: {svm_test_score:.4f}")

# Weibull AFT Model Coefficients
aft_fitter.print_summary()

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_offense_date',
       'c_arrest_date', 'c_days_from_arrest', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_charge_degree',
       'r_days_from_arrest', 'r_offense_date', 'r_arrest_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'is_violent_recid',
       'vr_charge_degree', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'duration'],
      dtype='object')


Survival SVM - Training Score: 0.6172
Survival SVM - Testing Score: 0.5291


<lifelines.WeibullAFTFitter: fitted with 14652 total observations, 14001 right-censored observations>
             duration col = 'duration'
                event col = 'event'
   number of observations = 14652
number of events observed = 651
           log-likelihood = -4273.25
         time fit was run = 2025-02-13 06:57:36 UTC

---
                      coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                     
lambda_ age           0.05      1.05      0.01            0.04            0.07                1.04                1.07
        decile_score  0.10      1.10      0.03            0.03            0.16                1.03                1.17
        priors_count  0.01      1.01      0.02           -0.03            0.04                0.97                1.04
        Intercept     5.94    378.93      0.39            5.17            6.71              175.18              819.68
rho_    Intercept    -0.66      0.52      0.03           -0.72           -0.61                0.49                0.55

                      cmp to      z      p  -log2(p)
param   covariate                                   
lambda_ age             0.00   6.08 <0.005     29.67
        decile_score    0.00   2.93 <0.005      8.21
        priors_count    0.00   0.34   0.73      0.45
        Intercept       0.00  15.08 <0.005    168.34
rho_    Intercept       0.00 -23.61 <0.005    406.93
---
Concordance = 0.56
AIC = 8556.49
log-likelihood ratio test = 54.78 on 3 df
-log2(p) of ll-ratio test = 36.93